In [26]:
!pip install U openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Users/dusisarathchandra/Documents/Projects/workspaces/zoomcamp-project/cloud-service-helper/env/bin/python3 -m pip install --upgrade pip' command.


In [27]:
import os, sys
import lancedb
from lancedb.table import Table
from openai import OpenAI


In [28]:
from retrieval import RetrieveContext
llm_client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')

### Get the context based on the query through vector search using LanceDB

In [29]:
q = "query that makes it easy to analyze data from S3"

result_context_df = RetrieveContext().get_context(q)
result_context_df

Getting context from the table
table name: vector_db
first 5 data:                                          description  \
0  An interactive query service that makes it eas...   
1  A managed service in the AWS Cloud that makes ...   
2  A web service that makes it easy to process la...   
3  A managed service that makes it easy to deploy...   
4  A cloud big data platform for processing vast ...   

                                  vector_description cloud_provider  \
0  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   
1  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   
2  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   
3  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   
4  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   

                        service   category  
0                 Amazon Athena  Analytics  
1            Amazon CloudSearch  Analytics  
2      Amazon Elastic MapReduce  A

,description,vector_description,cloud_provider,service,category,_relevance_score
0,An interactive query service that makes it eas...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Athena,Analytics,0.8
1,An interactive query service that makes it eas...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Athena,Analytics,0.8
2,A managed service in the AWS Cloud that makes ...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon CloudSearch,Analytics,0.8
3,A web service that makes it easy to process la...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Elastic MapReduce,Analytics,0.8
4,A managed service that makes it easy to deploy...,"[0.00044248256, -0.0026783892, -0.12139637, 0....",aws,Amazon Elasticsearch Service,Analytics,0.8


In [30]:
selected_columns_df = result_context_df[['description', 'category', 'service', 'cloud_provider']]
selected_columns_df.to_dict(orient='records')

[{'description': 'An interactive query service that makes it easy to analyze data in Amazon S3 using standard SQL.',
  'category': 'Analytics',
  'service': 'Amazon Athena',
  'cloud_provider': 'aws'},
 {'description': 'An interactive query service that makes it easy to analyze data in Amazon S3 using standard SQL.',
  'category': 'Analytics',
  'service': 'Amazon Athena',
  'cloud_provider': 'aws'},
 {'description': 'A managed service in the AWS Cloud that makes it simple to set up manage and scale a search solution for your website or application.',
  'category': 'Analytics',
  'service': 'Amazon CloudSearch',
  'cloud_provider': 'aws'},
 {'description': 'A web service that makes it easy to process large amounts of data efficiently.',
  'category': 'Analytics',
  'service': 'Amazon Elastic MapReduce',
  'cloud_provider': 'aws'},
 {'description': 'A managed service that makes it easy to deploy operate and scale Elasticsearch clusters in the AWS Cloud.',
  'category': 'Analytics',
  's

In [31]:
def build_prompt(query, search_results):
    prompt_template = """
You're a cloud asset category finder. Answer the DESCRIPTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the DESCRIPTION. Generate a short answer by saying `Service likely falls under the category: and LIKELY SERVICE IS:` 
while returing response consider sending only `answer` & full name of the `service`.

DESCRIPTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"answer: {doc['category']}\nservice: {doc['service']}\ndescription: {doc['description']}\ncloud_provider: {doc['cloud_provider']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

## Create a build prompt with context

In [32]:
prompt = build_prompt(q, selected_columns_df.to_dict(orient='records'))
print(prompt.strip())

You're a cloud asset category finder. Answer the DESCRIPTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the DESCRIPTION. Generate a short answer by saying `Service likely falls under the category: and LIKELY SERVICE IS:` 
while returing response consider sending only `answer` & full name of the `service`.

DESCRIPTION: query that makes it easy to analyze data from S3

CONTEXT: 
answer: Analytics
service: Amazon Athena
description: An interactive query service that makes it easy to analyze data in Amazon S3 using standard SQL.
cloud_provider: aws

answer: Analytics
service: Amazon Athena
description: An interactive query service that makes it easy to analyze data in Amazon S3 using standard SQL.
cloud_provider: aws

answer: Analytics
service: Amazon CloudSearch
description: A managed service in the AWS Cloud that makes it simple to set up manage and scale a search solution for your website or application.
cloud_provider: aws

answer: Analytics
service: Amaz

In [33]:
def ollama_llm(prompt):
    response = llm_client.chat.completions.create(
        model='gemma2:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

## Feed the prompt to llm to generate results

In [34]:
result = ollama_llm(prompt=prompt)
print(result.strip())

Service likely falls under the category: **Analysis** 
and LIKELY SERVICE IS: Amazon Athena


## Adding everything to a RAG flow

In [35]:
def rag(query):
    search_results = RetrieveContext().get_context(query)[['description', 'category', 'service', 'cloud_provider']].to_dict(orient='records')
    print("**************************************")
    prompt = build_prompt(query, search_results)
    print(prompt)
    answer = ollama_llm(prompt)
    return answer

In [36]:
query = "analyze genomic data."
response = rag(query)
print(response.strip())

Getting context from the table
table name: vector_db
first 5 data:                                          description  \
0  An interactive query service that makes it eas...   
1  A managed service in the AWS Cloud that makes ...   
2  A web service that makes it easy to process la...   
3  A managed service that makes it easy to deploy...   
4  A cloud big data platform for processing vast ...   

                                  vector_description cloud_provider  \
0  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   
1  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   
2  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   
3  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   
4  [0.00044248256, -0.0026783892, -0.12139637, 0....            aws   

                        service   category  
0                 Amazon Athena  Analytics  
1            Amazon CloudSearch  Analytics  
2      Amazon Elastic MapReduce  A